In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import re
import os
import itertools
print(os.listdir("../input"))
import emoji
import pickle
# Any results you write to the current directory are saved as output.
DEBUG = False

['glove-twitter-100d', 'jigsaw-unintended-bias-in-toxicity-classification']


In [2]:
tqdm.pandas(desc="")
class process_description(object):
    def __init__(self,vec_dim = 128):
        self.model = None
        self.max_len = 0
        self.unknown_vect = None
        self.padding_vect = np.zeros([100,])
        self.word2vec_dict = {}
        self.word2vec_vocab = {}
        self.vec_dim = vec_dim
    def process_description_file(self,file,max_len =  0.99):
        print('\nProcessing text data...')
        file = file.fillna('').str.lower()
        #file = file.progress_apply(emoji.demojize)
        #file = file.progress_apply(lambda x:re.sub('_',' ',x))        
        file = file.progress_apply(lambda x: re.sub('[0-9][0-9a-z]*', '<number>', x))
        file = file.progress_apply(lambda x: re.findall(r"\w+|<\w+>|[^\w\s:]", x, re.UNICODE))
        text_len = file.apply(len)
        self.max_len = int(np.ceil(text_len.quantile(0.99)))
        print('The length of sentences: ',self.max_len)
        print('Data processed sucessfully.')
        return file
    def train_word2vec(self,file,word2vec_epochs = 10,min_count = 1,similar_words = None):
        print('\nTraning word2vec model...')
        start_time = time.time()
        self.model = Word2Vec(file, 
                              iter=word2vec_epochs, 
                              min_count=min_count , 
                              size=self.vec_dim,
                              window=5,
                              workers = 4)
        self.model.save('word2vec.model')
        self.word2vec_dict = self.model.wv
        self.word2vec_vocab = self.model.wv.vocab
        self.unknown_vect = np.mean(self.model.wv.syn0,axis = 0)
        print('Training done. Time used:',time.time()-start_time)
        if similar_words:
            print(self.model.most_similar(str(similar_words),topn =5))
    def padding_file(self,file):
        file = pad_sequences(file,
            maxlen = self.max_len,
            dtype=object,
            padding = 'post',
            truncating = 'post',
            value = 'PADDING')
        return file

    def token2vec(self,token):
        if token is 'PADDING':
            return self.padding
        elif token in self.word2vec_vocab:
            return self.word2vec_dict[token]
        else:
            return self.unknown_vect
    def sent2vec(self,sentence):
        return np.array(list(map(self.token2vec,sentence)))
    def text_to_vect(self,file):
        print('\nConverting sentences to vectors')
        sent_vector = []
        start_time = time.time()
        for sentence in tqdm(file):
            sent_vector.append(self.sent2vec(sentence))
        #sent_vector = np.array(sent_vector)
        print('Converted sucessfully. Time used:',time.time()-start_time)
        return np.array(sent_vector)
#desc_parser = process_description()
#desc_parser.process_description_file(pd.Series(['1123 is d0! 💙', '2thd the main we! 2o3 to do'] ))  

## Loading GloVe files

In [3]:
glove_embedding_path = "../input/glove-twitter-100d/glove.twitter.27B.100d.txt" 
glove_embedding_index = {}
f = open(glove_embedding_path)

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_embedding_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(glove_embedding_index))

Found 1193514 word vectors.


## Loading train and test files and pre-process text

In [4]:
if DEBUG:
    nrows = 10000
else:
    nrows = None
    
train = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv",usecols = ['id','comment_text','target'],nrows = nrows)
test = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv",usecols = ['id','comment_text'], nrows = nrows)

In [5]:
train_0 = train[train.target == 0]
train_non_0 = train[train.target != 0]

del train
# clean duplicates
print(len(train_0))
train_0 = train_0.drop_duplicates(subset = ['comment_text'])
print(len(train_0))

print(len(train_non_0))
train_non_0 = train_non_0.drop_duplicates(subset = ['comment_text','target'])
print(len(train_non_0))
# merge text
total_text = pd.concat([train_0,train_non_0,test],ignore_index = True)
print(total_text.shape)

#process data
desc_parser = process_description()
tokens = desc_parser.process_description_file(total_text['comment_text'])

1264764
1246819
540110
539202
(1883341, 3)

Processing text data...


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
100%|██████████| 1883341/1883341 [01:03<00:00, 29584.03it/s]


The length of sentences:  217
Data processed sucessfully.


In [6]:
total_text.head()

,comment_text,id,target
0,"This is so cool. It's like, 'would you want yo...",59848,0.0
1,Thank you!! This would make my life a lot less...,59849,0.0
2,This is such an urgent design problem; kudos t...,59852,0.0
3,Is this something I'll be able to install on m...,59855,0.0
4,FFFFUUUUUUUUUUUUUUU,59863,0.0


In [7]:
total_text.tail()

,comment_text,id,target
1883336,That's the thing...it's called a plan. Get in...,7097315,NaN
1883337,"It's not quite the way you describe it, Mike. ...",7097316,NaN
1883338,What right have you to criticise? You have no ...,7097317,NaN
1883339,My concern is that if China allies with the US...,7097318,NaN
1883340,"Dear Young Philip, General Kelly can speak for...",7097319,NaN


In [8]:

total_tokens = list(itertools.chain.from_iterable(tokens))
unique_tokens = set(total_tokens)
len_unique_tokens = len(unique_tokens)
len_unique_tokens = len(unique_tokens)
len_glove = len(glove_embedding_index)
len_tokens = len(total_tokens)
ood = {}
ind = {}
within = 0
for token in tqdm(total_tokens):
    if token in  glove_embedding_index:
        within += 1
        if token in ind:
            ind[token] += 1
        else:
            ind[token] = 1
            
    else:
        if token in ood:
            ood[token] += 1
        else:
            ood[token] =1
len_ood = len(ood)
len_ind = len(ind)
useless_glove = set(glove_embedding_index.keys()).difference(set(ind.keys()))
print("{} of unique tokens were embedded.\n{} of text were embeded\n{} of Glove keys were usesless".format(len_ind/(len_unique_tokens), within/len_tokens, len(useless_glove)/len_glove) )


100%|██████████| 118889003/118889003 [01:55<00:00, 1032428.73it/s]


0.35879453560335206 of unique tokens were embedded.
0.9935801631711891 of text were embeded
0.9102825773304712 of Glove keys were usesless


In [9]:
ood = {k: v for k, v in sorted(ood.items(), key=lambda x: x[1],reverse = True)}
list(ood.items())[:50]

[('\xad', 17386),
 ('murkowski', 3570),
 ('siemian', 2180),
 ('brexit', 2041),
 ('notley', 1707),
 ('tillerson', 1585),
 ('theglobeandmail', 1421),
 ('manafort', 1396),
 ('drumpf', 1288),
 ('gorsuch', 1203),
 ('alceste', 1150),
 ('magisterium', 1093),
 ('sloter', 1034),
 ('usccb', 1012),
 ('gabbard', 1002),
 ('kealoha', 961),
 ('tfsa', 959),
 ('eweb', 942),
 ('utm_term', 938),
 ('trumpster', 889),
 ('trumpcare', 869),
 ('\u200b', 841),
 ('chaput', 782),
 ('imua', 754),
 ('klastri', 754),
 ('softwood', 743),
 ('bigly', 724),
 ('wavemaker', 701),
 ('fptp', 664),
 ('massengill', 662),
 ('shannyn', 659),
 ('punahou', 640),
 ('ibbitson', 600),
 ('trumpers', 598),
 ('anwr', 595),
 ('trumpism', 589),
 ('rhyner', 573),
 ('monsef', 569),
 ('clericalism', 568),
 ('zinke', 566),
 ('😂', 565),
 ('coghill', 562),
 ('wohlforth', 538),
 ('fluoridated', 537),
 ('gasline', 527),
 ('kellyanne', 527),
 ('hanabusa', 524),
 ('mushers', 514),
 ('pfds', 504),
 ('_r', 501)]

## Creating embedding layers weights

In [10]:
print(len(ind))
# if the word not in the embedding, using <unk> instead. The vector of <unk> is the mean of all vectors, the <pad> keeps 0
word_index = {'<pad>':0,'<unk>':1}
for i,word in enumerate(ind.keys()):
    word_index[word] = i+2
print("Number of unique tokens: ",len(word_index))

107079
Number of unique tokens:  107081


In [11]:
EMBEDDING_DIM = glove_embedding_index['i'].shape[0]
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = glove_embedding_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_matrix[1] = embedding_matrix.mean(axis = 0)
print(embedding_matrix.shape)

(107082, 100)


In [12]:
f = open("word_index.pkl","wb")
pickle.dump(word_index,f)
f.close()

pd.DataFrame(embedding_matrix).to_csv('embedding_weights_twitter_100.csv', index=False,header = False)

In [13]:
'''
emoji_dict = {}
emojis = ['😂', '😁', '🔥', '🌮', '😉', '😀', '🆘', '👍', '😊', '🙄', '😄', '😃','😆', '👎','😜', '😡', '🤔',
'🌯', '😎', '💨', '🤣', '💥', '💙']
for e in emojis:
    emoji_dict[e] = re.sub(":","",re.sub("_"," ",emoji.demojize(e)))
emoji_dict

from textblob import TextBlob
correct_word = []
for word in tqdm(ood.keys()):
    corrected_word,coef = Word(word).spellcheck()[0]
    if word != corrected_word:
        correct_word.append([word,corrected_word,coef])   
'''

'\nemoji_dict = {}\nemojis = [\'😂\', \'😁\', \'🔥\', \'🌮\', \'😉\', \'😀\', \'🆘\', \'👍\', \'😊\', \'🙄\', \'😄\', \'😃\',\'😆\', \'👎\',\'😜\', \'😡\', \'🤔\',\n\'🌯\', \'😎\', \'💨\', \'🤣\', \'💥\', \'💙\']\nfor e in emojis:\n    emoji_dict[e] = re.sub(":","",re.sub("_"," ",emoji.demojize(e)))\nemoji_dict\n\nfrom textblob import TextBlob\ncorrect_word = []\nfor word in tqdm(ood.keys()):\n    corrected_word,coef = Word(word).spellcheck()[0]\n    if word != corrected_word:\n        correct_word.append([word,corrected_word,coef])   \n'